# goal of notebook
- test direct matching and create a list for a quick manual match with closest publisher candidates with Levenstein
- convert manual matches to v2 (because changed BTLF extraction from publisher to imprint)
    -> insert manual_match_v2 dict into ExtractionGraphBTLF
- use again if editors change a lot and need to match again OR adding a new source

In [1]:
from rdflib import Graph
from rdflib import RDF
import sys
sys.path.append('../')

sys.path.append('../andre')
import andre.utils as utils
from andre.utils import schema as SCHEMA
import collections

# editor alignment BLTF/ BNF_Constellations

In [2]:
publishers_btlf_graph = Graph()
publishers_btlf_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_btlf_graph.parse("../final_datasets/btlf_books/Graphes/grapheEditeurs_Imprint_andre.ttl")
print(len(list(publishers_btlf_graph.subjects(RDF.type, SCHEMA.Publisher))))


publishers_bnf_constellations_graph = Graph()
publishers_bnf_constellations_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_bnf_constellations_graph.parse("../final_datasets/publisher_BNF_Constellation_NoDuplicate.ttl")
len(list(publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher)))

1022


2228

In [3]:
import Levenshtein

def find_top_x_similar_strings_levenshtein(string_list, target_string, top_x=5):
    distances = []
    for string in string_list:
        distance = Levenshtein.distance(string, target_string)
        if distance <= 8:
            distances.append((string, distance))
    distances.sort(key=lambda x: x[1])
    if distances:
         return distances[:top_x]
    else: 
        return None

In [4]:
publishers_bnf_constellations_dict = {}

for publisher in publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher):
    raw_name = str(publishers_bnf_constellations_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)
    
    if preprocessed_name not in publishers_bnf_constellations_dict.keys(): # don't add doublons
        # doublons[preprocessed_name]+= publisher
        publishers_bnf_constellations_dict[preprocessed_name] = utils.Publisher(source="bnf_constellations", 
                                                                        uri=publisher, 
                                                                        raw_name=raw_name, 
                                                                        preprocessed_name=preprocessed_name)
    else:
        print("duplicate found: check source file or eliminate")

In [5]:
len(publishers_btlf_graph)

2044

# manual matches conversion from v1 to v2 
####  v1 (key = raw_name_bltf, value = uri_bnf_constellations) 
#### v2 (key = raw_name_btlf, value = raw_name_bnf_constellations)
## reason for change: uri_bnf_constellation depends on raw_name_bnf_constellations

In [6]:
def find_raw_name_by_uri(target_uri):
    for preprocessed_name in publishers_bnf_constellations_dict.keys():
        uri = publishers_bnf_constellations_dict[preprocessed_name].uri.replace("http://schema.org/", "")
        if uri == target_uri:
            return publishers_bnf_constellations_dict[preprocessed_name].raw_name
    return "not found"

In [7]:
find_raw_name_by_uri("Gulf_Stream")

'Gulf Stream'

In [8]:
manual_matches = {"Gulf Stream (Editions du)": "Gulf_Stream", 
                  "Magnard-Vuibert": "Magnard",
                  "Musée du Louvre": "MuseduLouvred",
                 "Soleil productions": "Soleilproduction",
                 "Editions Ouest-France-Edilarge": "OuestFrance",
                 "Plume de carotte": "Plumecarotte",
                 "Editions Dadoclem": "Dadoclemd",
                 "Jean-Claude Gawsewitch éditeur": "JCGawsewitch",
                 "Editions de L'Archipel": "LArchipel",
                 "Odile Jacob": "OJacob",
                 "Editions du Centre Pompidou": "CentrePompidou",
                 "Musées de la ville de Strasbourg": "lesMusesdelaVilledeStrasbourg",
                 "Lancosme multimédia": "Lancosmed",
                 "Critères": "Critresd",
                 "Quespire": "Quespired",
                 "Les éditions du Bonhomme vert": "leBonhommevert",
                 "Vengeur masqué": "Le_Vengeur_Masqué",
                 "Editions Ecole active": "Ecole_Active",
                 "Editions [MiC_MaC]": "MicMac",
                 "Eveil et découvertes": "Eveiletdcouverte",
                 "Editions des Grandes personnes": "Les_Grandes_Personnes",
                 "Editions-Diffusion Karthala": "Karthala",
                 "Les éditions Carpentier": "DCarpentier",
                 "Pascal Galodé éditeurs": "PGalod",
                 "Prisma Media": "Prisma",
                 "Art à la page": "lArtlapage",
                 "CERCIL": "duCercil",
                 "Les Editions de la Grande ourse": "La_Grande_Ourse",
                 "Editions de La Différence": "La_Différence",
                 "Tertium": "Tertiumd",
                 "Editions de la Renarde rouge": "La_Renarde_Rouge",
                 "Aniss éditions": "Anissd",
                 "Editions du Rouergue": "Rouergue",
                 "Les éditions du soleil de minuit": "Soleil_De_Minuit",
                 "Editions des Arènes": "Les_Arènes",
                 "Le Square éditeur": "leSquared",
                 "Griffon bleu": "leGriffonbleu",
                 "Les éditions du Kilowatt": "Kilowatt",
                 "Ophelbe": "Ofelbe",
                 "Editions Privat SAS": "Privat",
                 "Losange": "duLosange",
                 "Les Editions Sandawe": "Sandawecom",
                 "Nouvelles éditions ivoiriennes": "NEINouvellesditionsIvoiriennes",
                 "Studio des Trois Becs": "StudiolesTroisBecs",
                 "Editions Imago": "AuzasditeursImago",
                 "205 Corp": "205fr",
                 "Librairie Téqui (Pierre)": "Téqui",
                 "Editions de la Coopérative": "laCooprative",
                 "Les Productions Somme toute": "Somme_toute",
                 "Réseau Canopé": "Canopditions",
                 "Editions Maelström": "maelstrmreEvolution",
                 "Le Dilettante": "Le_dilletante",
                 "Les Editions de Minuit": "Minuit",
                 "Editions Jean-Paul Gisserot": "JPGisserot",
                 "Les éditions du dernier havre": "Le_dernier_havre",
                 "Éditions du Septentrion": "Septentrion",
                 "Orca Book Publishing": "Orca",
                 "LéR - Les éditeurs réunis": "Les_Éditeurs_Réunis",
                 "Presses de l'Université de Montréal": "Les_Presses_De_Luniversité_De_Montréal",
                 "Éditions du Noroit": "Le_Noroît",
                 "Editions de Fallois": "Fallois",
                 "6 pieds sous terre éditions": "Six_Pieds_Sous_Terre",
                 "Editions du Triomphe": "Triomphe",
                 "Éditions du Vermillon": "Vermillon",
                 "Éditions ADA inc.": "AdA",
                 "Homme sans nom (Editions de l')": "LHomme_Sans_Nom",
                 "Editions des Mille saisons": "Mille_Saisons",
                 "Vincze (Eva)": "Vincze",
                 "Edizioni Corraini": "Corraini",
                 "Editions Picquier (Philippe)": "PPicquier",
                 "Ed. de l'Homme sans nom":"LHomme_Sans_Nom",
                 "Jaguar":"duJaguar",
                 "Palissade":"La_Palissade",
                 "Note de partage":"uneNotedepartage",
                 "Privat SAS": "Privat",
                 "Marmaille et compagnie":"Marmaillecompagnie",
                 "Alain Gorius": "alManarAGorius",
                 "Harmattan Guinée": "lHarmattanGuine",
                 "les Ed. Ago": "AGO",
                 "Louvre éditions": "MuseduLouvred",
                 "Calicot": "leCalicot",
                 "Aventuriers de l'étrange": "lesAventuriersdeltrange",
                 "Revue dessinée":"LaRevuedessine",
                 "Diplodocus": "leDiplodocus",
                 "Nouv. éd. ivoiriennes":"NEINouvellesditionsIvoiriennes",
                 "Museum d'histoire naturelle": "Musumnationaldhistoirenaturelle",
                 "Musée cantonal des beaux-arts de Lausanne": "MusedesbeauxartsdeLausanne",
                 "Editions Deux-Cent-Cinq": "205fr",
                 "Inédite": "lIndite",
                 }

In [9]:
manual_matches_v2 = {}
for match_key in manual_matches:
    manual_matches_v2[match_key] = find_raw_name_by_uri(manual_matches[match_key])
manual_matches_v2

{'Gulf Stream (Editions du)': 'Gulf Stream',
 'Magnard-Vuibert': 'Magnard',
 'Musée du Louvre': 'Musée du Louvre éd.',
 'Soleil productions': 'Soleil production',
 'Editions Ouest-France-Edilarge': 'Ouest-France',
 'Plume de carotte': 'Plume carotte',
 'Editions Dadoclem': 'Dadoclem éd.',
 'Jean-Claude Gawsewitch éditeur': 'J.-C. Gawsewitch',
 "Editions de L'Archipel": "L'Archipel",
 'Odile Jacob': 'O. Jacob',
 'Editions du Centre Pompidou': 'Centre Pompidou',
 'Musées de la ville de Strasbourg': 'les Musées de la Ville de Strasbourg',
 'Lancosme multimédia': 'Lancosme éd.',
 'Critères': 'Critères éd.',
 'Quespire': 'Quespire éd.',
 'Les éditions du Bonhomme vert': 'le Bonhomme vert',
 'Vengeur masqué': 'Le Vengeur Masqué',
 'Editions Ecole active': "L'École active",
 'Editions [MiC_MaC]': 'Mic-Mac',
 'Eveil et découvertes': 'Eveil et découverte',
 'Editions des Grandes personnes': 'Les Grandes Personnes',
 'Editions-Diffusion Karthala': 'Karthala',
 'Les éditions Carpentier': 'D. Carp

In [10]:
manual_matches_v2 = {
'Gulf Stream (Editions du)': 
   'Gulf Stream',
'Magnard-Vuibert': 
   'Magnard',
'Musée du Louvre': 
   'Musée du Louvre éd.',
'Soleil productions': 
   'Soleil production',
'Editions Ouest-France-Edilarge': 
   'Ouest-France',
'Plume de carotte': 
   'Plume carotte',
'Editions Dadoclem': 
   'Dadoclem éd.',
'Jean-Claude Gawsewitch éditeur': 
   'J.-C. Gawsewitch',
"Editions de L'Archipel": 
   "L'Archipel",
'Odile Jacob': 
   'O. Jacob',
'Editions du Centre Pompidou': 
   'Centre Pompidou',
'Musées de la ville de Strasbourg': 
   'les Musées de la Ville de Strasbourg',
'Lancosme multimédia': 
   'Lancosme éd.',
'Critères': 
   'Critères éd.',
'Quespire': 
   'Quespire éd.',
'Les éditions du Bonhomme vert': 
   'le Bonhomme vert',
'Vengeur masqué': 
   'Le Vengeur Masqué',
'Editions Ecole active': 
   "L'École active",
'Editions [MiC_MaC]': 
   'Mic-Mac',
'Eveil et découvertes': 
   'Eveil et découverte',
'Editions des Grandes personnes': 
   'Les Grandes Personnes',
'Editions-Diffusion Karthala': 
   'Karthala',
'Les éditions Carpentier': 
   'D. Carpentier',
'Pascal Galodé éditeurs': 
   'P. Galodé',
'Prisma Media': 
   'Prisma',
'Art à la page': 
   "l'Art à la page",
'CERCIL': 
   'du Cercil',
'Les Editions de la Grande ourse': 
   'La Grande Ourse',
'Editions de La Différence': 
   'La Différence',
'Tertium': 
   'Tertium éd.',
'Editions de la Renarde rouge': 
   'La Renarde Rouge',
'Aniss éditions': 
   'Aniss éd.',
'Editions du Rouergue': 
   'Rouergue',
'Les éditions du soleil de minuit': 
   'Soleil De Minuit',
'Editions des Arènes': 
   'Les Arènes',
'Le Square éditeur': 
   'le Square éd.',
'Griffon bleu': 
   'le Griffon bleu',
'Les éditions du Kilowatt': 
   'Kilowatt',
'Ophelbe': 
   'Ofelbe',
'Editions Privat SAS': 
   'Privat',
'Losange': 
   'du Losange',
'Les Editions Sandawe': 
   'Sandawe.com',
'Nouvelles éditions ivoiriennes': 
   'NEI - Nouvelles éditions Ivoiriennes',
'Studio des Trois Becs': 
   'Studio les Trois Becs',
'Editions Imago': 
   'Auzas éditeurs-Imago',
'205 Corp': 
   '205.fr',
'Librairie Téqui (Pierre)': 
   'Téqui',
'Editions de la Coopérative': 
   'la Coopérative',
'Les Productions Somme toute': 
   'Somme toute',
'Réseau Canopé': 
   'Canopé éditions',
'Editions Maelström': 
   'maelstrÖm reEvolution',
'Le Dilettante': 
   'Le dilletante',
'Les Editions de Minuit': 
   'Minuit',
'Editions Jean-Paul Gisserot': 
   'J.-P. Gisserot',
'Les éditions du dernier havre': 
   'Le dernier havre',
'Éditions du Septentrion': 
   'Septentrion',
'Orca Book Publishing': 
   'Orca',
'LéR - Les éditeurs réunis': 
   'Les Éditeurs Réunis',
"Presses de l'Université de Montréal": 
   'Les Presses De L’université De Montréal',
'Éditions du Noroit': 
   'Le Noroît',
'Editions de Fallois': 
   'Fallois',
'6 pieds sous terre éditions': 
   'Six Pieds Sous Terre',
'Editions du Triomphe': 
   'Triomphe',
'Éditions du Vermillon': 
   'Vermillon',
'Éditions ADA inc.': 
   'AdA',
"Homme sans nom (Editions de l')": 
   "L'Homme Sans Nom",
'Editions des Mille saisons': 
   'Mille Saisons',
'Vincze (Eva)': 
   'É. Vincze',
'Edizioni Corraini': 
   'Corraini',
'Editions Picquier (Philippe)': 
   'P. Picquier',
"Ed. de l'Homme sans nom": 
   "L'Homme Sans Nom",
'Jaguar': 
   'du Jaguar',
'Palissade': 
   'La Palissade',
'Note de partage': 
   'une Note de partage',
'Privat SAS': 
   'Privat',
'Marmaille et compagnie': 
   'Marmaille & compagnie',
'Alain Gorius': 
   'al Manar-A. Gorius',
'Harmattan Guinée': 
   "l'Harmattan Guinée",
'les Ed. Ago': 
   'AGO',
'Louvre éditions': 
   'Musée du Louvre éd.',
'Calicot': 
   'le Calicot',
"Aventuriers de l'étrange": 
   "les Aventuriers de l'étrange",
'Revue dessinée': 
   'La Revue dessinée',
'Diplodocus': 
   'le Diplodocus',
'Nouv. éd. ivoiriennes': 
   'NEI - Nouvelles éditions Ivoiriennes',
"Museum d'histoire naturelle": 
   "Muséum national d'histoire naturelle",
'Musée cantonal des beaux-arts de Lausanne': 
   'Musée des beaux-arts de Lausanne',
'Editions Deux-Cent-Cinq': 
   '205.fr',
'Inédite': 
   "l'Inédite",
"Néva éd.":"Neva",
"Ed. des Mille saisons": 
   "Mille Saisons",
"Sorbier":"Le Sorbier",
"Quadrants":"Quadrant",
"Rouge et or":"Rouge & Or",
"Nuée bleue": 
   "la Nuée bleue",
"Rocher": 
   "du Rocher",
"EP Emmanuel Proust éditions": 
   "Emmanuel Proust",
"Petit Lézard": 
   "Le Petit lézard",
"Afrobul":
   "Afro bulles éd.",
"Atalante": 
   "L'Atalante",
"Cherche Midi": 
   "Le Cherche Midi",
"Courrier du livre": "le Courrier du livre",
"Cité des sciences & de l'industrie":
   "Cité des sciences et de l'industrie",
"Ecole des loisirs":
   "L'École Des Loisirs",
"De La Martinière Jeunesse":
   "La Martinière Jeunesse",
"Temps apprivoisé":
   "le Temps apprivoisé",
"Archipel":
   "L'Archipel",
"Ecole active":
   "L'École active",
"Blake et Mortimer":
   "Blake & Mortimer",
"les Ed. du jardin des mots":
   "du Jardin des mots",
"M. Lafon": 
   "Michel Lafon",
"les Ed. du bonhomme vert":
   "le Bonhomme vert",
"Fond du Tiroir (Le)":
   "le Fond du tiroir",
"Ed. des Grandes personnes":
   "Les Grandes Personnes",
"Éditions de la Bagnole":
   "La Bagnole",
"les Ed. Fei":
   "Fei",
"Tulipe noire":
   "la Tulipe noire",
"Joie de lire":
   "La Joie De Lire",
"Temps des cerises":
   "le Temps des cerises",
"Orbestier":
   "d'Orbestier",
"Ed. de la Gouttière":
   "La Gouttière",
"Éditions Hurtubise inc.":
   "Hurtubise",
"Ed. de la Renarde rouge":
   "La Renarde Rouge",
"S. Diantantu":
   "Diantantu Editions",
"Arche éditeur":
   "L'Arche",
"Rocher jeunesse":
   "Le Rocher Jeunesse",
"Éditions de l'Isatis":
   "Isatis",
"Agrume":
   "L'Agrume",
"Petites moustaches éditions (Les)":
   "les Petites moustaches",
"Square éditeur":
   "le Square éd.",
"Cosmographe":
   "le Cosmographe",
"Harmattan Mali":
   "l'Harmattan Mali",
"Tropique édition":
   "Tropiques",
"Ed. du Centre Pompidou":
   "Centre Pompidou",
"Lattès":
   "JC Lattès",
"Atelier des Noyers":
   "l'Atelier des noyers",
"Lac aux Fées":
   "le lac aux fées",
"Le Robert":
   "Robert",
"Éditions du Phoenix":
   "Du Phœnix",
"Saint-Jean":
   "Guy Saint-Jean",
"Éditions du Trécarré":
   "Trécarré",
"Humanoïdes associés":
   "Les Humanoïdes Associés",
"Beech Street Books (French)":
   "Beech Street Books",
"Ed. de Fallois":
   "Fallois",
"Pré-aux-Clercs":
   "Le Pré Aux Clercs",
"Éditions de l'Hexagone":
   "L'Hexagone",
"6 pieds sous terre":
   "Six Pieds Sous Terre",
"Ed. de l'Olivier":
   "L'Olivier"
 }
 
direct_match_counter = 0
manual_match_counter = 0
levenstein_match_counter = 0
no_match_counter = 0
total_counter = 0

btlf_publishers_dict = {}
matches_dict = {}
non_matches_dict = {}
doublons_btlf = []
for publisher in publishers_btlf_graph.subjects(RDF.type, SCHEMA.Publisher):
    # print(publisher)
    total_counter += 1
    raw_name = str(publishers_btlf_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    if preprocessed_name in btlf_publishers_dict.keys():
        doublons_btlf.append(preprocessed_name)
        
    btlf_publishers_dict[preprocessed_name] = utils.Publisher(source="btlf", 
                                                uri=publisher, 
                                                raw_name=raw_name, 
                                                preprocessed_name=preprocessed_name)

    if preprocessed_name in publishers_bnf_constellations_dict.keys():
        direct_match_counter +=1
        # print(preprocessed_name)
        matches_dict[preprocessed_name] = [btlf_publishers_dict[preprocessed_name], publishers_bnf_constellations_dict[preprocessed_name]]
    elif raw_name in manual_matches_v2.keys(): # don't change graph
        # but: avoir uri du truc matché
        manual_match_counter += 1
        matched_raw_name = manual_matches_v2[raw_name]
        # print(raw_name, "->", matched_uri)
        # print(SCHEMA[matched_uri])
        # print(find_preprocessed_name_by_uri(SCHEMA[matched_uri]))
    else:
        matched_key = find_top_x_similar_strings_levenshtein(publishers_bnf_constellations_dict.keys(), preprocessed_name)
        if matched_key:
            levenstein_match_counter += 1
            print("\""+ raw_name+ "\":")
            for key in matched_key:
                print("  ", "\""+ publishers_bnf_constellations_dict[key[0]].raw_name + "\",")
        else:
            non_matches_dict[preprocessed_name] = publisher
            no_match_counter += 1
print()
print("direct", direct_match_counter, "/", len(btlf_publishers_dict.keys()))
print("manual", manual_match_counter, "/", len(btlf_publishers_dict.keys()))
print("no match", no_match_counter, "/", len(btlf_publishers_dict.keys()))
print("levenstein", levenstein_match_counter, "/", len(btlf_publishers_dict.keys()))
print("total counter", total_counter)
print("doublons counter", len(doublons_btlf))
print("total number of keys in dict", total_counter - len(doublons_btlf))
print("sum = ", direct_match_counter  + manual_match_counter + no_match_counter + levenstein_match_counter, "==", len(doublons_btlf) + len(btlf_publishers_dict.keys()))

"Alter Comics":
   "AH Comics",
   "Bliss comics",
   "404 comics",
   "Alternatives",
   "DC Comics",
"Didier-Erudition":
   "A. Michel éducation",
   "Didier Jeunesse",
"Éditions Chouette":
   "Caurette",
   "Clochette",
   "Cheyne",
   "Chêne",
   "Palette",
"Les éditions d'Avallon":
   "Amalion",
   "Dalloz",
   "Pavilion",
   "Amazon",
   "Carlton",
"Éditions Logiques":
   "Tropiques",
   "B. Jacques",
   "L'Olivier",
   "Utopique",
   "afrique editions",
"Age d'homme":
   "de l'Homme",
   "à la gomme",
   "ADEME",
   "Emme",
   "Petit Homme",
"Boomerang éditeur jeunesse":
   "Bonneton jeunesse",
   "Baobab jeunesse",
   "Bréal jeunesse",
   "Leméac jeunesse",
   "Toucan jeunesse",
"Hugo Image":
   "Héritage",
   "Aréopage",
   "Auzas éditeurs-Imago",
   "Degliame",
   "Du Passage",
"PUF":
   "LGF",
   "Lux",
   "Oup",
   "PH",
   "PLB éd.",
"INRAP":
   "IRP",
   "CNRS éd.",
   "ENAG Editions",
   "Garae",
   "IMAV éditions",
"Un, deux, quatre":
   "le Square éd.",
   "Ne Bâté",
 

In [11]:
print()
print("direct", direct_match_counter, "/", len(btlf_publishers_dict.keys()))
print("manual", manual_match_counter, "/", len(btlf_publishers_dict.keys()))
print("no match", no_match_counter, "/", len(btlf_publishers_dict.keys()))
print("levenstein", levenstein_match_counter, "/", len(btlf_publishers_dict.keys()))
print("total counter", total_counter)
print("doublons counter", len(doublons_btlf))
print("total number of keys in dict", total_counter - len(doublons_btlf))
print("sum = ", direct_match_counter  + manual_match_counter + no_match_counter + levenstein_match_counter, "==", len(doublons_btlf) + len(btlf_publishers_dict.keys()))


direct 742 / 1015
manual 91 / 1015
no match 32 / 1015
levenstein 157 / 1015
total counter 1022
doublons counter 7
total number of keys in dict 1015
sum =  1022 == 1022


In [15]:
doublons_btlf # not eliminated because very few duplicates

['lesallusifs',
 'les400coups',
 'lemeac',
 'arte',
 'albinmicheljeunesse',
 'nordsud',
 'ventsdouest']

In [13]:
len(non_matches_dict)

32

In [14]:
non_matches_dict

{'canopecrdpdelacademiedeparis': rdflib.term.URIRef('http://schema.org/Publisher128'),
 'bretagneenvironnement': rdflib.term.URIRef('http://schema.org/Publisher150'),
 'maitrisedeshautsdeseine': rdflib.term.URIRef('http://schema.org/Publisher232'),
 'sncfdirectiondelarecherchedepartementmarketingetdeveloppement': rdflib.term.URIRef('http://schema.org/Publisher287'),
 'radiofranceinternationale': rdflib.term.URIRef('http://schema.org/Publisher298'),
 'canopecrdpdelacademiedaixmarseille': rdflib.term.URIRef('http://schema.org/Publisher301'),
 'associationculturellefranceamerique': rdflib.term.URIRef('http://schema.org/Publisher314'),
 'galeriebarbier&mathon': rdflib.term.URIRef('http://schema.org/Publisher316'),
 'museeduluxembourgdeparis': rdflib.term.URIRef('http://schema.org/Publisher334'),
 'lememorialdecaen': rdflib.term.URIRef('http://schema.org/Publisher363'),
 'chateaudeversailles': rdflib.term.URIRef('http://schema.org/Publisher395'),
 'mondeglobalnouvelles': rdflib.term.URIRef(